# Address Matching Deep Dive

This notebook experiments with different ways of comparing addresses in order to demonstrate the power of parsed address comparison.

In [1]:
import json
import logging
import os
import random
import re
import sys
import time
import warnings
from numbers import Number
from typing import Callable, Dict, List, Literal, Sequence, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pytest
import random
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from datasets import Dataset
from fuzzywuzzy import fuzz

from postal.parser import parse_address
from scipy.spatial import distance
from sklearn.metrics import (
    accuracy_score,
    precision_recall_curve,
    precision_recall_fscore_support,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    auc
)
from sklearn.model_selection import train_test_split
from sentence_transformers import InputExample, SentenceTransformer, SentencesDataset, SentenceTransformerTrainer, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction, BinaryClassificationEvaluator
from sentence_transformers.model_card import SentenceTransformerModelCardData
from sentence_transformers.training_args import BatchSamplers, SentenceTransformerTrainingArguments
from tenacity import retry
from torch.utils.data import DataLoader
from torch.optim import RAdam
from tqdm.autonotebook import tqdm
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, TrainingArguments, Trainer
from transformers.integrations import WandbCallback

from utils import (
    augment_gold_labels,
    compute_sbert_metrics,
    compute_classifier_metrics,
    format_dataset,
    gold_label_report,
    preprocess_logits_for_metrics,
    structured_encode_address,
    tokenize_function,
    to_dict,
    save_transformer,
    load_transformer,
)

#### Pin Random Seeds for Reproducibility

In [2]:
RANDOM_SEED = 31337

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.mps.manual_seed(RANDOM_SEED)

#### Setup Basic Logging

In [3]:
logging.basicConfig(stream=sys.stderr, level=logging.ERROR)

logger = logging.getLogger(__name__)

#### Ignore Warnings

In [4]:
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)

#### Configure Weights & Biases

`wandb` needs some environment variables to work.

In [5]:
os.environ["WANDB_LOG_MODEL"] = "end"
os.environ["WANDB_WATCH"] = "gradients"
os.environ["WANDB_PROJECT"] = "libpostal-reborn"
os.environ["WANDB_DISABLED"] = "false"
os.environ["WANDB_IGNORE_GLOBS"] = ".env"

#### Optionally Disable `wandb` Uploads

Weights and Biases can be slow...

In [6]:
os.environ["WANDB_MODE"] = "online"

#### Configure Huggingface APIs

In [7]:
os.environ["HF_ENDPOINT"] = "https://huggingface.co/"

#### Configure Huggingface APIs

Squash any warnings...

In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#### Configure Pandas to Show More Rows

In [9]:
pd.set_option("display.max_rows", 40)
pd.set_option("display.max_columns", None)

### Use CUDA or MPS if Avaialable

CPU training and even inference with sentence transformers and deep learning models is quite slow. Since all machine learning in this library is based on [PyTorch](https://pytorch.org/get-started/locally/), we can assign all ML operations to a GPU in this one block of code. Otherwise we default to CPU without acceleration. The notebook is still workable in this mode, you just may need to grab a cup of tea or coffee while you wait for it to train the Sentence-BERT model below.

In [10]:
# Check for CUDA or MPS availability and set the device
if torch.backends.mps.is_available():
    device = torch.device("mps")
    logger.debug("Using Apple GPU acceleration")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    logger.debug("Using NVIDIA CUDA GPU acceleration")
else:
    device = "cpu"
    logger.debug("Using CPU for ML")

device

device(type='cuda')

### Use Weights & Biases for Logging Metrics

Weights & Biases has a free account for individuals with public projects. Using it will produce charts during our training runs that anyone can view. You can create your own project for this notebook and login with that key to log your own training runs.

You may need to run the following command from your shell before the next cell, otherwise you will have to paste your project key into the 

```bash
wandb login
```

In [11]:
# Login to wandb. Comment out if you already haven't via `wandb login` from a CLI
wandb.login()

wandb: Currently logged in as: rjurney. Use `wandb login --relogin` to force relogin


True

## Data Augmentation with the OpenAI GPT4o API

We need training data for our supervised learning approaches to addres matching. Open the sister notebook [Address Data Augmentation.ipynb](Address%20Matching%20Deep%20Dive.ipynb) before procceeding to further cells in order to create some training data via minimal manual labeling and programmatic data labeling for data augmentation. This will teach you programmatic data labeling, a critical skill that LLMs make MUCH EASIER because they understand things like the semantics of global addresses.

In [21]:
gold_df = pd.read_csv("data/gold.csv")

In [22]:
# If you want to start from here and not run the data augmentation pipeline again...
augment_results_df = pd.read_parquet("data/training.6.parquet")

augment_results_df.head(20)

,Address1,Address2,Description,Label
0,"123 E Main St, Springfield, IL 62701","123 East Main Street, Springfield, Illinois 62701",Different directional prefix formats for same ...,1.0
1,"456 W Elm St, Boston, MA 02118","456 West Elm Street, Boston, Massachusetts 02118",Different directional prefix formats for same ...,1.0
2,"789 S Oak St, Denver, CO 80203","789 South Oak Street, Denver, Colorado 80203",Different directional prefix formats for same ...,1.0
3,"321 N Pine St, Seattle, WA 98101","321 North Pine Street, Seattle, Washington 98101",Different directional prefix formats for same ...,1.0
4,"654 E Maple St, Austin, TX 73301","654 East Maple Street, Austin, Texas 73301",Different directional prefix formats for same ...,1.0
5,"987 W Cedar St, San Francisco, CA 94102","987 West Cedar Street, San Francisco, Californ...",Different directional prefix formats for same ...,1.0
6,"246 S Birch St, New York, NY 10001","246 South Birch Street, New York, New York 10001",Different directional prefix formats for same ...,1.0
7,"135 N Cedar Ave, Chicago, IL 60601","135 North Cedar Avenue, Chicago, Illinois 60601",Different directional prefix formats for same ...,1.0
8,"864 E Pine Ave, Los Angeles, CA 90001","864 East Pine Avenue, Los Angeles, California ...",Different directional prefix formats for same ...,1.0
9,"753 W Spruce St, Houston, TX 77001","753 West Spruce Street, Houston, Texas 77001",Different directional prefix formats for same ...,1.0


### Data Augmentation Complete!

Starting by hand labeling under 100 records and iterating a few times on data augmentation instructions for GPT4o, we have multiplied them by many times to get almost 10,000 synthetic records! This is enough to fine-tune a `SentenceTransformer` or semantic text similarity classifier model. GPT4o is a powerful tool for data augmentation! This can work for a variety of problems.

LLM based data augmentation is a powerful tool for your data labeling toolbox.

# Comparing Different Approaches to Address Matching

Now we're going to compare the following methods of address matching:

1) Database Lookups - we'l use [pycountry](https://pypi.org/project/pycountry/) ([github](https://github.com/pycountry/pycountry)) to improve international address matching (see [PyCountry Nation Matching](PyCountry%20Nation%20Matching.ipynb)).
2) Text Embeddings - we'll use transfer learning to load an existing [SentenceTransformer](https://sbert.net) model to sentence encode pairs of addresses to create fixed-length embeddings for each address and then compute a similarity score via [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). This won't work without fine-tuning, so we fine-tune the model to the task.
3) Deep Matching Model - We'll train a deep semantic textual similarity classification model based on a Siamese BERT network as defined in [Sentence-BERT](https://arxiv.org/abs/1908.10084) to classify address pairs as matching or not matching.

# Machine Learning Approaches to Address Matching

In this section we pursue two machine learning approaches to address matching, in order of sophistication. First we fine-tune a pre-trained embedding model to our task, try it on our data and search for a threshold similarity that results in good performance for our address matching problem. Second we build a Siamese BERT network model based on [Sentence-BERT](https://arxiv.org/abs/1908.10084) to classify pairs of addresses as match or mismatch. We will train it using the same dataset we use to fine-tune a sentence transformer, and if we have enough training data this will likely be a more powerful approach.

# Parsed Address Matching with Libpostal, PyTorch and `Cosine-Sentence-BERT`

Our next strategy will be to parse the addresses using Libpostal and then to encode them in a way that perserves the parsed staructure. We will use a deep network architecture called `Cosine-Sentence-BERT`, a derivative of `Sentence-BERT`, to build a classifier for pairs of addresses that can achieve better performance than fine-tuned sentence transformers and cosine similarity alone.

Embeddings as a solution to this problem have a side-effect of optimizing an embedding for information retrieval... but they ignore the structure of parsed addresses. A deep network that is aware of it can perform better. Let's try out an implementation of the Sentence-BERT model, which was outlined by Nils Reimers and Iryna Gurevych in the original paper that created sentence tranformers, [Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks
](https://arxiv.org/abs/1908.10084).

In [23]:
from typing import List, Tuple
from collections import defaultdict

def parse_standardize_address(address: str) -> str:
    """Parse with Libpostal, then stringify fields in standard order"""

    # Libpostal parse the address
    parsed_address: List[Tuple[str, str]] = parse_address(address)
    
    FIELD_ORDER = [
        "house_number",
        "house",
        "road",
        "unit",
        "level",
        "staircase",
        "entrance",
        "category",
        "near",
        "suburb",
        "city_district",
        "city",
        "island",
        "state_district",
        "state",
        "postcode",
        "po_box",
        "country_region",
        "country",
        "world_region"
    ]
    
    # Fields that typically precede a comma in addresses
    COMMA_AFTER = {"road", "city", "state", "country_region"}
    
    # Create a defaultdict to group values by field
    address_dict = defaultdict(list)
    for value, field in parsed_address:
        if value.strip():
            address_dict[field].append(value.strip())
    
    # Create a list of non-empty address components in the specified order
    address_components = []
    for field in FIELD_ORDER:
        if field in address_dict:
            component = ' '.join(address_dict[field])
            if field in COMMA_AFTER and field != FIELD_ORDER[-1]:
                component += ','
            address_components.append(component)
    
    # Manually join the components with a space unless they are null
    address_string = str()
    for address_component in address_components:
        if address_component and isinstance(address_component, str) and len(address_component) > 0 and address_component != ",":
            address_string += address_component + " "
    
    return address_string.strip()

In [24]:
train_df, tmp_df = train_test_split(augment_results_df, test_size=0.2, shuffle=True)
eval_df, test_df = train_test_split(tmp_df, test_size=0.5, random_state=42, shuffle=True)

# Encode the addresses using [COL] / [VAL] special characters
parsed_train_dataset = Dataset.from_dict({
    "sentence1": [parse_standardize_address(x) for x in train_df["Address1"].tolist()],
    "sentence2": [parse_standardize_address(x) for x in train_df["Address2"].tolist()],
    "label": train_df["Label"].tolist(),
})
train_dataset = Dataset.from_dict({
    "sentence1": train_df["Address1"].tolist(),
    "sentence2": train_df["Address2"].tolist(),
    "label": train_df["Label"].tolist(),
})

# Encode the addresses using [COL] / [VAL] special characters
parsed_eval_dataset = Dataset.from_dict({
    "sentence1": [parse_standardize_address(x) for x in eval_df["Address1"].tolist()],
    "sentence2": [parse_standardize_address(x) for x in eval_df["Address2"].tolist()],
    "label": eval_df["Label"].tolist(),
})
eval_dataset = Dataset.from_dict({
    "sentence1": eval_df["Address1"].tolist(),
    "sentence2": eval_df["Address2"].tolist(),
    "label": eval_df["Label"].tolist(),
})

# Encode the addresses using [COL] / [VAL] special characters
parsed_test_dataset = Dataset.from_dict({
    "sentence1": [parse_standardize_address(x) for x in test_df["Address1"].tolist()],
    "sentence2": [parse_standardize_address(x) for x in test_df["Address2"].tolist()],
    "label": test_df["Label"].tolist(),
})
test_dataset = Dataset.from_dict({
    "sentence1": test_df["Address1"].tolist(),
    "sentence2": test_df["Address2"].tolist(),
    "label": test_df["Label"].tolist(),
})

print(f"Training data:   {len(train_df):,}")
print(f"Validation data: {len(eval_df):,}")
print(f"Test data        {len(eval_df):,}")

Training data:   8,024
Validation data: 1,003
Test data        1,003


In [25]:
SBERT_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
VARIANT = "raw-embeddings"
MODEL_SAVE_NAME = ("Cosine-Sentence-BERT" + "-" + VARIANT).replace("/", "-")

EPOCHS = 6
BATCH_SIZE = 16
PATIENCE = 2
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.02
MODEL_OUTPUT_FOLDER = f"data/{MODEL_SAVE_NAME}"
SAVE_EVAL_STEPS = 100

LIBPOSTAL = True

### Substitute Libpostal Parsed `Datasets`

In [26]:
if LIBPOSTAL:
    logging.error("Using LIBPOSTAL dataset")
    train_dataset = parsed_train_dataset
    eval_dataset = parsed_eval_dataset
    test_dataset = parsed_test_dataset
else:
    logging.error("Using raw dataset")

ERROR:root:Using LIBPOSTAL dataset


In [27]:
# Initialize Weights & Biases
wandb.init(
    entity="rjurney",
    # set the wandb project where this run will be logged
    project="libpostal-reborn",
    # track hyperparameters and run metadata
    config={
        "model": "Cosine-Sentence-BERT",
        "variant": VARIANT,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "patience": PATIENCE,
        "learning_rate": LEARNING_RATE,
        "sbert_model": SBERT_MODEL,
        "model_output_folder": MODEL_OUTPUT_FOLDER,
        "save_eval_steps": SAVE_EVAL_STEPS,
        "model_save_name": MODEL_SAVE_NAME,
        "weight_decay": WEIGHT_DECAY,
        "libpostal": LIBPOSTAL,
    },
    save_code=True,
)

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer


class CosineSentenceBERT(nn.Module):
    def __init__(self, model_name=SBERT_MODEL, dim=384):
        super().__init__()
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

        # Update the FFNN to output embedding dimension
        self.ffnn = nn.Sequential(
            nn.Linear(dim, dim//2),
            nn.GELU(),
            nn.Dropout(0.1),
        )

    @staticmethod
    def mean_pool(token_embeds, attention_mask):
        in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float()
        pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(in_mask.sum(1), min=1e-9)
        return pool

    def encode(self, input_ids, attention_mask):
        outputs = self.model(input_ids, attention_mask=attention_mask)[0]
        embeddings = self.mean_pool(outputs, attention_mask)
        return self.ffnn(embeddings)

    def forward(self, input_ids_a, input_ids_b, attention_mask_a=None, attention_mask_b=None, labels=None, **kwargs):
        # Encode both sentences
        embed_a = self.encode(input_ids_a, attention_mask_a)
        embed_b = self.encode(input_ids_b, attention_mask_b)

        # Compute cosine similarity
        cosine_sim = F.cosine_similarity(embed_a, embed_b)

        loss = None
        if labels is not None:
            loss_fct = nn.CosineEmbeddingLoss()
            # CosineEmbeddingLoss expects 1 for similar pairs and -1 for dissimilar pairs
            loss = loss_fct(embed_a, embed_b, (labels * 2) - 1)

        return {"loss": loss, "similarity": cosine_sim}

    def predict(self, a: str, b: str):
        encoded_a = self.tokenizer(a, padding=True, truncation=True, return_tensors="pt")
        encoded_b = self.tokenizer(b, padding=True, truncation=True, return_tensors="pt")
        
        with torch.no_grad():
            embed_a = self.encode(encoded_a["input_ids"].to(self.model.device), 
                                  encoded_a["attention_mask"].to(self.model.device))
            embed_b = self.encode(encoded_b["input_ids"].to(self.model.device), 
                                  encoded_b["attention_mask"].to(self.model.device))
            
            similarity = F.cosine_similarity(embed_a, embed_b).item()
            
            return similarity

In [29]:
classifier_model = CosineSentenceBERT()
tokenizer = classifier_model.tokenizer

In [30]:
tokenized_train_dataset = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
tokenized_eval_dataset = eval_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
tokenized_test_dataset = test_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

tokenized_train_dataset = format_dataset(tokenized_train_dataset)
tokenized_eval_dataset = format_dataset(tokenized_eval_dataset)
tokenized_test_dataset = format_dataset(tokenized_test_dataset)

Map:   0%|          | 0/8024 [00:00<?, ? examples/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments


class CosineSimilarityTrainer(Trainer):
    """Trainer for Cosine-Sentence-BERT. Uses RAdam optimizer and custom loss function."""
    
    def create_optimizer(self):
        return RAdam(self.model.parameters(), lr=self.args.learning_rate)

    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = outputs["loss"]
        return (loss, outputs) if return_outputs else loss


training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_FOLDER,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    run_name=MODEL_SAVE_NAME,
    load_best_model_at_end=True,
    save_steps=SAVE_EVAL_STEPS,
    eval_steps=SAVE_EVAL_STEPS,
    save_strategy="steps",
    eval_strategy="steps",
    greater_is_better=False,
    metric_for_best_model="eval_loss",
    weight_decay=0.01,
    logging_dir="./logs",
    optim="adamw_torch",
    fp16=True if device.type == "cuda" else False,
    data_seed=RANDOM_SEED,
)

optimizer = RAdam(classifier_model.parameters(), lr=training_args.learning_rate)

trainer = CosineSimilarityTrainer(
    model=classifier_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_classifier_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
    optimizers=(optimizer, None),
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,No log,0.324218,0.683948,0.769119,0.638452,0.967033
200,No log,0.266537,0.764706,0.815913,0.710598,0.957875
300,No log,0.246892,0.773679,0.819124,0.724965,0.941392
400,No log,0.218388,0.797607,0.840784,0.735254,0.981685


In [ ]:
print(f"Best model checkpoint path: {trainer.state.best_model_checkpoint}")

In [ ]:
# Evaluate the model
pd.DataFrame([trainer.evaluate()])

### Save the Best Model

Because we used `load_best_model_at_end=True`, our model is now the best one we fine-tuned. Save it.

In [ ]:
save_transformer(classifier_model, "data/classifier_model")

In [ ]:
wandb.finish()

### Testing the Model

In [ ]:
classifier_model = load_transformer(CosineSentenceBERT, "data/classifier_model", device=device)

In [ ]:
classifier_model.predict("3413 Sean Way, Lawrenceville, GA 30044", "3413 Sean Way, Lawrenceville, GA 30044")

In [ ]:
classifier_model.predict("101 Oak Ct.,", "101 Oak Street")

In [ ]:
classifier_model.predict("101 Oak Pl.", "101 Oak Place")

### Probability and Boolean Prediction Methods

In [ ]:
def classifier_match(row: pd.Series) -> pd.Series:
    """classifier_match - Sentence-BERT address matching, float output"""
    return classifier_model.predict(row["Address1"], row["Address2"])


def classifier_match_boolean(row: pd.Series, threshold=0.5) -> pd.Series:
    """classifier_match_binary - Sentence-BERT address matching, boolean output"""
    return 1 if classifier_model.predict(row["Address1"], row["Address2"]) > threshold else 0

### Synthetic Data Evaluation

In [ ]:
y_true = test_df["Label"]
y_scores = test_df.apply(classifier_match, axis=1)

In [ ]:
# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

# Compute F1 score for each threshold
f1_scores = [f1_score(y_true, y_scores >= t) for t in thresholds]

# Find the threshold that maximizes the F1 score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f'Best Threshold: {best_threshold}')
print(f'Best F1 Score: {best_f1_score}')

roc_auc = roc_auc_score(y_true, y_scores)
print(f'AUC-ROC: {roc_auc}')

# Create a DataFrame for Seaborn
pr_data = pd.DataFrame({
    'Precision': precision[:-1],
    'Recall': recall[:-1],
    'F1 Score': f1_scores
})

In [ ]:
# Plot Precision-Recall curve using Seaborn
sns.lineplot(data=pr_data, x='Recall', y='Precision', marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Augmented Test Set Precision-Recall Curve')
plt.show()

### Gold Label Evaluation

In [ ]:
y_true = gold_df["Label"]
y_scores = gold_df.apply(classifier_match, axis=1)

In [ ]:
# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

# Compute F1 score for each threshold
f1_scores = [f1_score(y_true, y_scores >= t) for t in thresholds]

# Find the threshold that maximizes the F1 score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f'Best Threshold: {best_threshold}')
print(f'Best F1 Score: {best_f1_score}')

roc_auc = roc_auc_score(y_true, y_scores)
print(f'AUC-ROC: {roc_auc}')

# Create a DataFrame for Seaborn
pr_data = pd.DataFrame({
    'Precision': precision[:-1],
    'Recall': recall[:-1],
    'F1 Score': f1_scores
})

In [ ]:
# Plot Precision-Recall curve using Seaborn
sns.lineplot(data=pr_data, x='Recall', y='Precision', marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Gold Label Precision-Recall Curve')
plt.show()

In [ ]:
raw_df, grouped_df = gold_label_report(
    gold_df,
    [
        # sbert_match_binary,
        classifier_match_boolean,
    ],
    threshold=best_threshold
)

In [ ]:
grouped_df

In [ ]:
# Truthiness analysis
correct_df = raw_df[raw_df["classifier_match_boolean_correct"]].reset_index(drop=True)
print(f"Number correct: {len(correct_df):,}")

correct_df.head(20)

In [ ]:
# Error analysis
wrong_df = raw_df[raw_df["classifier_match_boolean_correct"] == False].reset_index()
print(f"Number wrong: {len(wrong_df):,}")

wrong_df.head(20)

In [ ]:
classifier_model.predict(
    "101 Oak Lane, Atlanta, GA 30308",
    "102 Oak Lane, Atlanta, GA 30308",
)

In [ ]:
classifier_model.predict(
    "101 Oak Lane, Macon, GA 30308",
    "101 Oak Lane, Atlanta, GA 30408",
)

In [ ]:
classifier_model.predict(
    "101 Oak Lane, Atlanta, GA 30308",
    "101 Oak Ln., Atlanta, GA 30308",
)

In [ ]:
classifier_model.predict(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "1202 Oak Rd., Lawrenceville, GA 30304",
)

In [ ]:
classifier_model.predict(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "3413 Sean Way, Lawrenceville, GA 30044, USA",
)